# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a la produccion eléctrica de energias renovables y no renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto en principio es prácticamente inmediato, también vamos a centrarnos en la produccion de energia hidroelectrica para poder medir cuanto tiempo tarda en afectar a este tipo de energía las precipitaciones..

En base a esto tenemos varias cuestiones que queremos despejar:
- Que relación hay entre precipitaciones y el aumento de la generación de energia eléctrica hidráulica.
- Que relación hay entre horas de sol, temperatura y el aumento de la generación de energia eléctrica sólar.
- Que relación hay entre viento y el aumento de la generación de energia eléctrica eólica, esta relación esta condicionada por la temperatura, percipitaciones o horas de sol.
- Cuantos días tarda en aumentar la generación eléctrica de fuentes de energía renovables en función de los fenomenos metereológicos.
- Dados una prediccion meteorologica que valores de generación eléctrica tendremos para una fecha determinada.



Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Debido a la falta de datos a nivel diario de REE por provincia o comunidades autonomas, voy a centrar el analisis a nivel de sistema eléctrico (Peninsula, Baleares, Canarias ,Ceuta y Melilla).

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Requisitos para la ejecución del notebook:

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request.

Además las versiones de cada libreria utilizada en este proyecto son:

El modelo ha utilizar al tratarse de una prediccion númerica y no de obtener una etiqueta, sera una regresión.
Para poder llevar a cabo esa regresion se han pasado los valores de fechas a númericos, y se ha creado una columna por cada tipo de energia, creando varias variables objetivo.



In [75]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en el directorio ../API que este notebook


with open('../API/API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

###  Lectura datos AEMET

Para la lectura de datos metereólogicos, vamos a utilizar la libreria aemet, de la que utilizaremos los metodos de las clases Aemet y Estacion para obtener los datos a nivel diario de cada estación meterologica para un rago de fechas, de las provincias que nos interesan para el estudio.

In [76]:
# Obtenemos el json de estaciones de mediciones de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)

In [77]:
from tqdm import tqdm_notebook as tqdm

# Definimos funciones que vamos a utilizar para leer los datos de AEMET
def estaciones_prov (prov,lista_estaciones):
    '''Dada una lista de provincias y un json de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    prov=list(map(str.upper, prov))
    for estacion in tqdm(lista_estaciones):
        if estacion['provincia'] in prov:
            lista_id.append(estacion['indicativo'])
    return lista_id


def lectura_diaria_json(date_ini,date_end,estaciones):
    '''Dado un json de la clase Estacion de la libreria Aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    valores_error=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in tqdm(estaciones):
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas por minuto.
            #Volvemos a intentar leer el dato que ha lanzado la excepcion.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicador'])
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;


def lectura_diaria_lista(date_ini,date_end,lista_estaciones):
    '''Dado una lista de id de estaciones de aemet, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in tqdm(lista_estaciones):
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(56) # para evitar errores por nº de lecturas.
            try:
                valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element)
                if type(valores_estacion)!=dict:
                    valores_diarios.extend(valores_estacion)
            except:
                print('Valor no encontrado')
                
    return valores_diarios;

In [80]:
# Parametros para lectura de datos de AEMET

provincias=['Ceuta','Melilla','Illes Balears','Las Palmas','Sta. Cruz de Tenerife']

date_ini="2018-01-01T00:00:00UTC"
date_end="2020-12-31T00:00:00UTC"

id_estaciones=estaciones_prov(provincias,info_estaciones)

# Pasamos los datos ha dataframes para su procesado y limpieza
estaciones=pd.DataFrame(info_estaciones)
df_weather=pd.DataFrame(lectura_diaria_lista(date_ini,date_end,id_estaciones),dtype=str)

In [81]:
df_weather

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2018-01-01,5000C,CEUTA,CEUTA,87,"15,6","0,0","11,5",07:30,"19,7",13:20,33,"1,7","7,5",00:10,"7,6","1024,3",11,"1021,2",Varias
1,2018-01-02,5000C,CEUTA,CEUTA,87,"18,0","0,0","13,3",06:50,"22,7",12:40,34,"2,2","8,9",15:20,"7,7","1023,5",10,"1020,1",15
2,2018-01-03,5000C,CEUTA,CEUTA,87,"18,0","0,0","14,2",Varias,"21,9",14:20,30,"1,1","8,6",18:30,"7,7","1022,1",10,"1018,3",24
3,2018-01-04,5000C,CEUTA,CEUTA,87,"16,4","0,0","14,5",21:50,"18,4",03:20,30,"4,2","18,6",13:40,"7,7","1018,4",00,"1014,0",24
4,2018-01-05,5000C,CEUTA,CEUTA,87,"16,2","9,8","14,0",23:40,"18,4",11:50,24,"3,3","11,4",18:20,"4,2","1014,0",00,"996,2",24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32475,2020-12-27,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"19,2","0,0","16,3",23:51,"22,0",12:54,99,"3,3","7,8",12:30,"5,3","1008,1",Varias,"1005,4",17
32476,2020-12-28,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"18,5","0,0","14,4",07:19,"22,6",Varias,99,"4,2","8,3",Varias,"7,6","1007,7",Varias,"1005,1",16
32477,2020-12-29,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"18,8","0,0","15,9",08:16,"21,6",11:44,99,"6,1","10,8",Varias,"8,5","1008,6",Varias,"1006,0",04
32478,2020-12-30,C429I,TENERIFE SUR AEROPUERTO,STA. CRUZ DE TENERIFE,64,"18,6","3,2","15,6",07:05,"21,7",13:49,10,"5,6","10,3",Varias,"6,7","1010,7",Varias,"1007,0",05


### Limpieza datos AEMET

In [82]:
# Creo una funcion que pase los indicadores a float y rellene los valores vacios por la temperatura media de las estaciones de esa provincia
def rellena_nulos_provincia(df,cols):
    # Defino un DataFrame vacio para acumular el resultado
    df_all=pd.DataFrame()
    
    # Hago un bucle para cada provincia del DataFrame de entrada
    for prov in df['provincia'].unique():
        df_prov=df[df['provincia']==prov].copy()
        
        #Para cada elemento de las columnas que nos interesan reemplazo , por ., paso a numerico y relleno los nulos por la media de la provincia
        for element in cols:
            df_prov[element]=df_prov[element].str.replace(',', '.')
            df_prov[element]=pd.to_numeric(df_prov[element],errors='coerce')
            df_prov[element].fillna(df_prov[element].mean(skipna=True),inplace=True)
        df_all=df_all.append(df_prov)
    return df_all

In [83]:
# Elimino las columnas que no me interesan
df_weather.drop(columns=['altitud','horaPresMax','horaPresMin','horaracha','dir','horatmin','horatmax','racha'],inplace=True)

# Limpio de nulos la muestra
df_weather=rellena_nulos_provincia(df_weather,df_weather.columns[5:])


In [84]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32480 entries, 0 to 32479
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       32480 non-null  object 
 1   indicativo  32480 non-null  object 
 2   nombre      32480 non-null  object 
 3   provincia   32480 non-null  object 
 4   altitud     32480 non-null  object 
 5   tmed        32480 non-null  float64
 6   prec        32480 non-null  float64
 7   tmin        32480 non-null  float64
 8   tmax        32480 non-null  float64
 9   velmedia    32480 non-null  float64
 10  racha       32480 non-null  float64
 11  sol         32480 non-null  float64
 12  presMax     32480 non-null  float64
 13  presMin     32480 non-null  float64
dtypes: float64(9), object(5)
memory usage: 3.7+ MB


In [89]:
# Dejo los datos agrupados por provincia y fecha
weather_grouped=df_weather.groupby(['fecha','provincia'],as_index=False)
df_weather_prov=weather_grouped.mean(['tmed','prec','tmin','tmax','velmedia','sol','presMax','presMin'])

## Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [88]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('../Data/REGION_REE',header=0,index_col='Region')

# Me quedo solo con los distintos sistemas electricos existentes
region_system=region_ree[region_ree['geo_limit']!='ccaa']

region_system

,geo_limit,geo_id
Region,,
peninsular,peninsular,8741
canarias,canarias,8742
baleares,baleares,8743
ceuta,ceuta,8744
melilla,melilla,8745


In [22]:
# Obtenemos los datos de REE a traves de su API. 

def lectura_ree_electric_system(d_inicio,d_fin,geo_id):
    
    # meter esto en una funcion con su try-exception    
    # Dividir la lectura por años
    
    geo_limit=region_system[region_system['geo_id']==geo_id]['geo_limit']
    
    parametros={'start_date':date_ini,
            'end_date':date_end,
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':geo_limit[0],
            'geo_ids':geo_id}
    
    URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

    ree_gen=requests.get(URL_GEN,params=parametros)
    
    df_ree=pd.DataFrame()
    for i in range(20):
        try:
            df=pd.json_normalize(ree_gen.json()['included'][i]['attributes'],meta=['title','type'],record_path=['values'])
            df['system']=geo_limit[0]
            df_ree=df_ree.append(df)
        except:
            pass #Cuando no hay datos para mas tecnologías
    df_ree.reset_index(inplace=True,drop=True)
    
    return df_ree

In [ ]:
df_ree_system=pd.DataFrame()

In [26]:

date_ini="2016-01-01T00:00:00UTC"
date_end="2016-12-31T00:00:00UTC"

for electric_system in tqdm(region_system['geo_id']):
    df_ree_system=df_ree_system.append(lectura_ree_electric_system(date_ini,date_end,electric_system))
df_ree_system   

,value,percentage,datetime,title,type,system
0,145184.069,0.268689,2020-01-01T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
1,159162.626,0.236886,2020-01-02T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
2,151987.615,0.233950,2020-01-03T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
3,139863.037,0.219290,2020-01-04T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
4,126791.218,0.219375,2020-01-05T00:00:00.000+01:00,Hidráulica,Renovable,peninsular
...,...,...,...,...,...,...
1786,580.347,1.000000,2016-12-27T00:00:00.000+01:00,Generación total,Generación total,melilla
1787,583.772,1.000000,2016-12-28T00:00:00.000+01:00,Generación total,Generación total,melilla
1788,575.986,1.000000,2016-12-29T00:00:00.000+01:00,Generación total,Generación total,melilla
1789,578.435,1.000000,2016-12-30T00:00:00.000+01:00,Generación total,Generación total,melilla


# Limpieza de datos de REE

In [53]:
# Renombro los campos
df_ree_system.rename(columns={'value':'Generacion_Mwh','title':'Tecnologia','type':'Renov_norenov'},inplace=True)

# Elimino la columna percentage por ser una columna generada de Generacion_Mwh.
df_ree_system.drop('percentage',axis=1,inplace=True)

# Cambio los valores nulos de Generacion en Mwh por 0
df_ree_system['Generacion_Mwh'].fillna(0)

# Elimino las filas para las cuales la fecha es nula y paso la fecha a formato corto.
df_ree_system['fecha']=df_ree_system['datetime'].str[:10]
df_ree_system=df_ree_system[~df_ree_system['fecha'].isna()]
df_ree_system.drop('datetime',axis=1,inplace=True)
df_ree_system

,Generacion_Mwh,Tecnologia,Renov_norenov,system,fecha
0,145184.069,Hidráulica,Renovable,peninsular,2020-01-01
1,159162.626,Hidráulica,Renovable,peninsular,2020-01-02
2,151987.615,Hidráulica,Renovable,peninsular,2020-01-03
3,139863.037,Hidráulica,Renovable,peninsular,2020-01-04
4,126791.218,Hidráulica,Renovable,peninsular,2020-01-05
...,...,...,...,...,...
1786,580.347,Generación total,Generación total,melilla,2016-12-27
1787,583.772,Generación total,Generación total,melilla,2016-12-28
1788,575.986,Generación total,Generación total,melilla,2016-12-29
1789,578.435,Generación total,Generación total,melilla,2016-12-30


# Repositorio de información

Para evitar tener que leer todos los datos en cada ejecucion, guardamos los datos de 2016 a 2020 en formato csv desde los DataFrame de REE y AEMET

In [54]:
df_ree_system.to_csv('../Data/ree_system.csv')

In [90]:
df_weather_prov.to_csv('../Data/wheater.csv')

# Análisis de la muestra

1 Evolución de temperaturas

2 Correlaciones

3 Evolución de precipitaciones

4 Datos estacionales

5 Localizar datos anomalos


,Generacion_Mwh,Tecnologia,Renov_norenov,system,fecha
0,145184.069,Hidráulica,Renovable,peninsular,2020-01-01
1,159162.626,Hidráulica,Renovable,peninsular,2020-01-02
2,151987.615,Hidráulica,Renovable,peninsular,2020-01-03
3,139863.037,Hidráulica,Renovable,peninsular,2020-01-04
4,126791.218,Hidráulica,Renovable,peninsular,2020-01-05
...,...,...,...,...,...
95,50648.539,Hidráulica,Renovable,peninsular,2020-04-05
96,80450.163,Hidráulica,Renovable,peninsular,2020-04-06
97,83006.635,Hidráulica,Renovable,peninsular,2020-04-07
98,78324.315,Hidráulica,Renovable,peninsular,2020-04-08


# Modelado 

Para hacer un primer modelo solo me voy a quedar con generacion total y melilla

In [103]:
df_elect=df_ree_system[(df_ree_system['Tecnologia']=='Generación total') 
              & (df_ree_system['system']=='melilla')
              & (df_ree_system['fecha'].str[:4]>='2018')][['fecha','Generacion_Mwh']]
df_elect.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096 entries, 1400 to 1781
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fecha           1096 non-null   object 
 1   Generacion_Mwh  1096 non-null   float64
dtypes: float64(1), object(1)
memory usage: 25.7+ KB


In [102]:
df_w=df_weather_prov[df_weather_prov['provincia']=='MELILLA']
df_w.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096 entries, 3 to 5478
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fecha      1096 non-null   object 
 1   provincia  1096 non-null   object 
 2   tmed       1096 non-null   float64
 3   prec       1096 non-null   float64
 4   tmin       1096 non-null   float64
 5   tmax       1096 non-null   float64
 6   velmedia   1096 non-null   float64
 7   racha      1096 non-null   float64
 8   sol        1096 non-null   float64
 9   presMax    1096 non-null   float64
 10  presMin    1096 non-null   float64
dtypes: float64(9), object(2)
memory usage: 102.8+ KB


In [105]:
df_w.merge(df_elect, on='fecha',how='left')

,fecha,provincia,tmed,prec,tmin,tmax,velmedia,racha,sol,presMax,presMin,Generacion_Mwh
0,2018-01-01,MELILLA,16.5,0.0,13.6,19.4,2.8,13.9,8.3,1028.3,1024.3,483.271
1,2018-01-02,MELILLA,17.6,0.0,11.6,23.6,6.4,14.2,8.8,1026.5,1022.2,558.697
2,2018-01-03,MELILLA,18.2,0.0,13.9,22.4,1.9,11.7,8.8,1026.6,1022.9,560.678
3,2018-01-04,MELILLA,17.1,0.0,12.8,21.4,2.8,13.1,8.8,1023.0,1017.8,562.873
4,2018-01-05,MELILLA,14.6,0.0,11.0,18.2,4.2,10.6,8.5,1017.8,1002.4,556.607
...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2020-12-27,MELILLA,11.4,0.0,6.4,16.4,2.2,7.5,8.8,1016.2,1005.4,544.484
1092,2020-12-28,MELILLA,16.0,3.0,12.4,19.5,3.6,11.7,7.7,1005.4,1001.6,568.881
1093,2020-12-29,MELILLA,13.2,1.6,11.1,15.2,3.6,14.4,3.0,1008.0,1003.4,585.858
1094,2020-12-30,MELILLA,12.0,0.0,8.6,15.3,2.5,11.9,7.4,1013.0,1007.8,589.612


# Interfaz y ploteado
